In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

/Users/juniverse/opt/anaconda3/envs/veda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.1
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [83 lines of output]
      Compiling without OpenMP...
      Compiling without OpenMP...
      Compiling without OpenMP...
      Compiling without OpenMP...
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-x86_64-cpython-38
      creating build/lib.macosx-10.9-x86_64-cpython-38/torch_scatter
      copying torch_scatter/placeholder.py -> build/lib.macosx-10.9-x86_64-cpython-38/torch_scatter
      copying torch_scatter/__init__.py -> build/lib.macosx-10.9-x86_64-cpython-38/torch_scatter
      copying torch_scatter/segment_csr.py -> build/lib.macosx-10.9-x86_64-cpython-38/torch_scatter
      copying torch_scatter/segment_coo.py -> build/lib.macosx-10.9-x86_64-cpython-38/torch_scatter
      copying torch_scatter/utils.py -> build/lib.macosx-10.9-x86_64-cpython-38/torch_scatter
      cop

In [ ]:
import torch_geometric
from torch_geometric.datasets import Planetoid

1️⃣  Data Handling of Graphs

In [ ]:
use_cuda_if_available = False

In [ ]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0,1,1,2],
                           [1,0,2,1]], dtype = torch.long) # (2,4) 크기의 행렬 : 4개의 엣지

x = torch.tensor([[-1],[0],[1]], dtype = torch.float) # (3,1) 크기의 행렬 : 3개의 노드

data = Data(x=x, edge_index = edge_index)

In [ ]:
data

Data(x=[3, 1], edge_index=[2, 4])

In [ ]:
# 엣지를 노드의 순서쌍으로 표현한 경우

edge_index = torch.tensor([[0,1],
                           [1,0],
                           [1,2],
                           [2,1]], dtype = torch.long) # 4개의 엣지

x = torch.tensor([[-1],[0],[1]], dtype = torch.float)

data = Data(x=x, edge_index = edge_index.t().contiguous())

data

Data(x=[3, 1], edge_index=[2, 4])

In [ ]:
# 함수

print(data.keys) # 해당 속성 이름

['edge_index', 'x']


In [ ]:
print(data['x'])

tensor([[-1.],
        [ 0.],
        [ 1.]])


In [ ]:
for key, item in data :
  print(f'{key} found in data')
  print(f'{item} found in data')
  print()

x found in data
tensor([[-1.],
        [ 0.],
        [ 1.]]) found in data

edge_index found in data
tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]]) found in data



In [ ]:
'edge_attr' in data  # 엣지 특징 행렬이 있냐 - 없다.

False

In [ ]:
data.num_nodes # 노드의 개수

3

In [ ]:
data.num_edges # 엣지의 개수

4

In [ ]:
data.num_node_features  # 노드 특성의 개수

1

In [ ]:
data.has_isolated_nodes() # 고립 노드 있는지 여부

False

In [ ]:
data.has_self_loops() # 자기 자신으로 화살표가 돌아오는 노드가 있는지 여부

False

In [ ]:
data.is_directed() # 단방향 그래프인지 여부

False

In [ ]:
# Transfer data object to GPU.
device = torch.device('cuda')
data = data.to(device) # GPU 사용으로 변경

2️⃣  Common Benchmark Datasets

* 효소 데이터셋 예제

In [ ]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root = '/tmp/ENZYMES', name = 'ENZYMES')
dataset

Extracting /tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...
Done!


ENZYMES(600)

In [ ]:
print(len(dataset)) # 그래프 개수
print(dataset.num_classes) # 그래프 클래스 수
print(dataset.num_node_features) # 노드의 특징 수

600
6
3


In [ ]:
data = dataset[0] # 인덱스 슬라이싱을 통해 데이터 확인하기
print(data)

# edge_index=[2, 168] : 84 개의 엣지 (2 : 양방향, 엣지 개수 : 84개)
#  x=[37, 3]  : 37개의 노드와 3개의 노드 특성 (하나의 노드가 3개의 값을 가짐)
#  y=[1]   : 그래프 레벨 타겟

Data(edge_index=[2, 168], x=[37, 3], y=[1])


In [ ]:
data.is_undirected() # 양방향 그래프인가요 - 네

True

In [ ]:
train_dataset = dataset[:540]
test_dataset = dataset[540:]

print(train_dataset)
print(test_dataset)



ENZYMES(540)
ENZYMES(60)


In [ ]:
dataset = dataset.shuffle() # 데이터셋 셔플
print(dataset)

ENZYMES(600)


* 논문 데이터셋 예제

In [ ]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root = 'tmp/Cora', name = 'Cora')

In [ ]:
print(len(dataset)) # 데이터셋 전체가 하나의 그래프임

print(dataset.num_classes)  # 7개의 클래스

print(dataset.num_node_features) # 1433개의 노드 특성 (하나의 노드에 1433개의 값이 존재)

1
7
1433


In [ ]:
data = dataset[0]

data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [ ]:
print(data.is_undirected())  # 양방향 그래프

True


In [ ]:
data.train_mask.sum().item() # 학습하기 위해 사용하는 노드

140

In [ ]:
data.val_mask.sum().item() # 검증 시 사용하는 노드

500

In [ ]:
data.test_mask.sum().item()  # 테스트 시 사용하는 노드

1000

3️⃣ Mini-batches

In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root = '/tmp/ENZYMES', name = 'ENZYMES',use_node_attr=True)

loader = DataLoader(dataset, batch_size = 32, shuffle = True)

for batch in loader :

  print(batch)

  print(batch.num_graphs)

DataBatch(edge_index=[2, 3938], x=[984, 21], y=[32], batch=[984], ptr=[33])
32
DataBatch(edge_index=[2, 3480], x=[902, 21], y=[32], batch=[902], ptr=[33])
32
DataBatch(edge_index=[2, 4264], x=[1154, 21], y=[32], batch=[1154], ptr=[33])
32
DataBatch(edge_index=[2, 4162], x=[1128, 21], y=[32], batch=[1128], ptr=[33])
32
DataBatch(edge_index=[2, 3646], x=[1036, 21], y=[32], batch=[1036], ptr=[33])
32
DataBatch(edge_index=[2, 4030], x=[1043, 21], y=[32], batch=[1043], ptr=[33])
32
DataBatch(edge_index=[2, 4480], x=[1253, 21], y=[32], batch=[1253], ptr=[33])
32
DataBatch(edge_index=[2, 4240], x=[1141, 21], y=[32], batch=[1141], ptr=[33])
32
DataBatch(edge_index=[2, 3882], x=[958, 21], y=[32], batch=[958], ptr=[33])
32
DataBatch(edge_index=[2, 3550], x=[946, 21], y=[32], batch=[946], ptr=[33])
32
DataBatch(edge_index=[2, 4462], x=[1138, 21], y=[32], batch=[1138], ptr=[33])
32
DataBatch(edge_index=[2, 4546], x=[1258, 21], y=[32], batch=[1258], ptr=[33])
32
DataBatch(edge_index=[2, 4032], x=[1

4️⃣  Data Transforms

* ShapeNet dataset 예제

In [ ]:
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root = '/tmp/ShapeNet', categories = ['Airplane'])

dataset[0]

Extracting /tmp/ShapeNet/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Processing...
Done!


Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])

In [ ]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

# 그래프 변환

dataset = ShapeNet(root = '/tmp/ShapeNet', categories = ['Airplane'],
                   pre_transform = T.KNNGraph(k=6),
                   transform = T.RandomTranslate(0.01))


In [ ]:
dataset[0]

Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])

5️⃣  Learning Methods on Graphs

In [ ]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module) :
  def __init__(self) :
    super(Net, self).__init__()
    self.conv1 = GCNConv(dataset.num_node_features, 16)
    self.conv2 = GCNConv(16, dataset.num_classes)

    # 2개의 GCNConv layer

  def forward(self, data) :
    x, edge_index = data.x, data.edge_index

    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = F.dropout(x, training = self.training)
    x = self.conv2(x, edge_index)

    return F.log_softmax(x, dim=1)

In [ ]:
# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)

# 옵티마이저 생성
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay = 5e-4)

# 훈련

model.train() # 학습 준비

for epoch in range(200) :
  optimizer.zero_grad() # 파라미터 초기화
  out = model(data) # 예측값
  loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask]) # 손실함수 계산
  loss.backward() # 역전파
  optimizer.step() # 파라미터 업데이터


In [ ]:
# 모델 평가

model.eval()

_, pred = model(data).max(dim=1)

correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()

print('정확도 : {:.4f}'.format(acc))

정확도 : 0.8040
